In [ ]:
!pip install torch torchvision torchaudio
!pip install opencv-python scikit-image tqdm pyyaml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

### Classification

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blank1508/mendeley-lbc-cervical-cancer")

print("Path to dataset files:", path)

100%|██████████| 2.06G/2.06G [01:45<00:00, 20.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/blank1508/mendeley-lbc-cervical-cancer/versions/2


In [ ]:
# prompt: i want to visualize the structure of directories of the above kaggle dataset

import os

def visualize_directory_structure(root_dir):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        level = dirpath.replace(root_dir, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f'{indent}{os.path.basename(dirpath)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in filenames:
            print(f'{subindent}{f}')

# Example usage (replace with your actual dataset path)
dataset_path = "/root/.cache/kagglehub/datasets/blank1508/mendeley-lbc-cervical-cancer/versions/2"  # Replace with the actual path
visualize_directory_structure(dataset_path)


2/
    Squamous cell carcinoma/
        SCC_4 (6).jpg
        scc_1 (18).jpg
        scc_1 (8).jpg
        SCC_3 (8).jpg
        SCC_3 (6).jpg
        SCC_3 (10).jpg
        scc_2 (14).jpg
        scc_1 (13).jpg
        SCC_3 (7).jpg
        SCC_3 (4).jpg
        scc_1 (14).jpg
        scc_1 (12).jpg
        scc_2 (15).jpg
        scc_2 (19).jpg
        scc_1 (11).jpg
        SCC_4 (11).jpg
        SCC_4 (14).jpg
        scc_2 (10).jpg
        scc_2 (6).jpg
        scc_2 (5).jpg
        scc_1 (1).jpg
        scc_1 (7).jpg
        scc_2 (13).jpg
        scc_2 (16).jpg
        scc_2 (7).jpg
        scc_2 (24).jpg
        scc_1 (15).jpg
        scc_2 (4).jpg
        SCC_4 (4).jpg
        SCC_3 (5).jpg
        scc_1 (20).jpg
        scc_2 (11).jpg
        SCC_4 (1).jpg
        scc_2 (22).jpg
        scc_1 (10).jpg
        SCC_4 (9).jpg
        scc_2 (9).jpg
        SCC_4 (8).jpg
        scc_2 (3).jpg
        scc_2 (21).jpg
        scc_2 (12).jpg
        scc_1 (5).jpg
        SCC_4 (10).jpg

In [ ]:
#Data downloading directly from Kaggle and data preparation for giving to DenseNet and ResNet
!pip install Pillow
import os
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from torchvision.models import resnet34
import torch.optim as optim
from sklearn.model_selection import KFold

# prompt: in above code ['ASC_H', 'ASC_US', 'HSIL', 'LSIL', 'NILM'] this should have label as 0,1,2,3,4 respectively

# ... (Your existing code)

# Data paths and parameters
data_dir = "/root/.cache/kagglehub/datasets/blank1508/mendeley-lbc-cervical-cancer/versions/2"
image_size = 224  # Adjust as needed
batch_size = 64
num_epochs = 30
learning_rate = 1e-4

# Define label mapping
label_mapping = {'Low squamous intra-epithelial lesion': 0, 'Negative for Intraepithelial malignancy': 1, 'High squamous intra-epithelial lesion': 2, 'Squamous cell carcinoma': 3}

# Image transformations
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.RandomHorizontalFlip(),  # Random Flip Augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ... (Your existing code)

# Load data
all_images = []
all_labels = []

for class_name in ['Low squamous intra-epithelial lesion','Negative for Intraepithelial malignancy','High squamous intra-epithelial lesion','Squamous cell carcinoma']:
    class_path = os.path.join(data_dir, class_name)
    for filename in os.listdir(class_path):
        # Construct the full image path directly
        image_path = os.path.join(class_path, filename)
        # Check if it's a file and if it has a valid image extension
        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp')):
            img = Image.open(image_path).convert("RGB")
            img_transformed = transform(img)
            all_images.append(img_transformed)
            all_labels.append(label_mapping[class_name])

In [ ]:
   # Iterate over files in the class directory



# ... (Rest of your code)

In [ ]:
all_images

[tensor([[[-0.5596, -0.5938, -0.6109,  ..., -0.9363, -0.9534, -0.9192],
          [-0.5424, -0.5596, -0.5938,  ..., -0.8849, -0.9192, -0.9192],
          [-0.5424, -0.5596, -0.5938,  ..., -0.8849, -0.9192, -0.9020],
          ...,
          [-0.4226, -0.4226, -0.3883,  ..., -0.8164, -0.7822, -0.7479],
          [-0.4397, -0.4054, -0.4054,  ..., -0.8164, -0.7650, -0.7479],
          [-0.4054, -0.4226, -0.4054,  ..., -0.7650, -0.7650, -0.7650]],
 
         [[-0.4951, -0.5126, -0.5301,  ..., -0.8277, -0.8277, -0.7927],
          [-0.4951, -0.4951, -0.5126,  ..., -0.7577, -0.7577, -0.7752],
          [-0.4776, -0.4951, -0.5126,  ..., -0.7927, -0.7927, -0.8102],
          ...,
          [-0.4601, -0.4426, -0.4601,  ..., -0.6877, -0.6527, -0.6176],
          [-0.4776, -0.4951, -0.4776,  ..., -0.6877, -0.6527, -0.6352],
          [-0.4776, -0.4951, -0.4776,  ..., -0.6527, -0.6702, -0.6527]],
 
         [[-0.1312, -0.1312, -0.1312,  ..., -0.2532, -0.2358, -0.2184],
          [-0.1312, -0.1487,

In [ ]:


all_images = torch.stack(all_images)
all_labels = torch.tensor(all_labels)
# ... (Rest of your code)

In [ ]:
# DenseNet121 training on ELIMA Dataset
import os
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Define device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Hyperparameters
batch_size = 64
num_epochs = 30
learning_rate = 1e-4

# 5-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_ids, test_ids) in enumerate(kfold.split(all_images)):
    print(f"\n----- Fold {fold+1} -----")

    # Sample data for current fold
    train_images = torch.stack([all_images[i] for i in train_ids])
    train_labels = torch.tensor([all_labels[i] for i in train_ids])
    test_images = torch.stack([all_images[i] for i in test_ids])
    test_labels = torch.tensor([all_labels[i] for i in test_ids])

    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = torch.utils.data.TensorDataset(test_images, test_labels)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize DenseNet-121 model
    model = models.densenet121(pretrained=True)  # Load pretrained DenseNet-121
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, 4)  # Modify classifier for 4 classes
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Loss: {running_loss/len(train_loader):.4f}')

    # Evaluation
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_targets, all_preds) * 100
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')

    # Print results
    print(f'Fold {fold+1} Accuracy: {accuracy:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')




Using device: cuda:0

----- Fold 1 -----


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 80.8MB/s]
Epoch 1/30: 100%|██████████| 13/13 [00:10<00:00,  1.26it/s]


Loss: 0.7570


Epoch 2/30: 100%|██████████| 13/13 [00:08<00:00,  1.57it/s]


Loss: 0.1433


...

Loss: 0.0435


Epoch 29/30: 100%|██████████| 13/13 [00:08<00:00,  1.61it/s]


Loss: 0.0250


Epoch 30/30: 100%|██████████| 13/13 [00:08<00:00,  1.61it/s]


Loss: 0.1331
Fold 5 Accuracy: 100.00%
Precision: 1.0000, Recall: 1.0000, F1-score: 1.0000


In [ ]:
# prompt: do the above same with same hyperparameters for resnet for the whole code, write the code even if it is same as above

# ... (Your existing code)

# Define device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Hyperparameters
batch_size = 64
num_epochs = 30
learning_rate = 1e-4

# 5-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_ids, test_ids) in enumerate(kfold.split(all_images)):
    print(f"\n----- Fold {fold+1} -----")

    # Sample data for current fold
    train_images = torch.stack([all_images[i] for i in train_ids])
    train_labels = torch.tensor([all_labels[i] for i in train_ids])
    test_images = torch.stack([all_images[i] for i in test_ids])
    test_labels = torch.tensor([all_labels[i] for i in test_ids])

    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = torch.utils.data.TensorDataset(test_images, test_labels)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize ResNet-34 model
    model = models.resnet34(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 4)  # Modify classifier for 4 classes
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Loss: {running_loss/len(train_loader):.4f}')

    # Evaluation
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_targets, all_preds) * 100
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')

    # Print results
    print(f'Fold {fold+1} Accuracy: {accuracy:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')


Using device: cuda:0

----- Fold 1 -----


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 192MB/s]
Epoch 1/30: 100%|██████████| 13/13 [00:04<00:00,  3.18it/s]


Loss: 0.5354


Epoch 2/30: 100%|██████████| 13/13 [00:04<00:00,  3.21it/s]


Loss: 0.1696


...

Loss: 0.3454


Epoch 29/30: 100%|██████████| 13/13 [00:04<00:00,  3.24it/s]


Loss: 0.2610


Epoch 30/30: 100%|██████████| 13/13 [00:04<00:00,  3.19it/s]


Loss: 0.1284
Fold 5 Accuracy: 100.00%
Precision: 1.0000, Recall: 1.0000, F1-score: 1.0000
